In [6]:
import pandas as pd
import numpy as np
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import * 
from textblob import TextBlob
import re
import dateutil.parser 
from html.parser import HTMLParser
import spacy

# Question 1: Jobposts Data Exploration and Analysis (Incomplete)
### The first step is to explore the data and to understand the sanity of data.

In [7]:
data_explore = pd.read_csv('data job posts.csv', parse_dates=['date'])
data_explore.head()

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

                                             jobpost          date  \
0  AMERIA Investment Consulting Company\r\nJOB TI...   Jan 5, 2004   
1  International Research & Exchanges Board (IREX...   Jan 7, 2004   
2  Caucasus Environmental NGO Network (CENN)\r\nJ...   Jan 7, 2004   
3  Manoff Group\r\nJOB TITLE:  BCC Specialist\r\n...   Jan 7, 2004   
4  Yerevan Brandy Company\r\nJOB TITLE:  Software...  Jan 10, 2004   

                                               Title  \
0                            Chief Financial Officer   
1  Full-time Community Connections Intern (paid i...   
2                                Country Coordinator   
3                                     BCC Specialist   
4                                 Software Developer   

                                           Company AnnouncementCode Term  \
0             AMERIA Investment Consulting Company              NaN  NaN   
1  International Research & Exchanges Board (IREX)              NaN  NaN   
2        Cauca

In [8]:
print(data_explore.dtypes)

jobpost             object
date                object
Title               object
Company             object
AnnouncementCode    object
Term                object
Eligibility         object
Audience            object
StartDate           object
Duration            object
Location            object
JobDescription      object
JobRequirment       object
RequiredQual        object
Salary              object
ApplicationP        object
OpeningDate         object
Deadline            object
Notes               object
AboutC              object
Attach              object
Year                 int64
Month                int64
IT                    bool
dtype: object


In [9]:
# hehe = pd.to_datetime("data_explore['date']", format="%b %d, %Y")
# hehe = data_explore.apply(parse(data_explore['date'], fuzzy=True))
# parse("Today is January 1, 2047 at 8:21:00AM", fuzzy=True) 
data_explore['date'] = [dateutil.parser.parse(x) for x in data_explore['date']]

In [10]:
data_explore['Year'].isnull().values.any()

False

In [11]:
data_explore.columns

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')

In [12]:
data_explore.count()

jobpost             19001
date                19001
Title               18973
Company             18994
AnnouncementCode     1208
Term                 7676
Eligibility          4930
Audience              640
StartDate            9675
Duration            10798
Location            18969
JobDescription      15109
JobRequirment       16479
RequiredQual        18517
Salary               9622
ApplicationP        18941
OpeningDate         18295
Deadline            18936
Notes                2211
AboutC              12470
Attach               1559
Year                19001
Month               19001
IT                  19001
dtype: int64

In [13]:
len(data_explore)

19001

In [14]:
data_explore.sort_values("date", inplace=True)
data_explore.drop_duplicates(keep=False, inplace=True)

## To check the amount of data that are removed (identical).

In [15]:
len(data_explore)

18923

In [16]:
data_explore.head(3)

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

                                             jobpost       date  \
0  AMERIA Investment Consulting Company\r\nJOB TI... 2004-01-05   
1  International Research & Exchanges Board (IREX... 2004-01-07   
2  Caucasus Environmental NGO Network (CENN)\r\nJ... 2004-01-07   

                                               Title  \
0                            Chief Financial Officer   
1  Full-time Community Connections Intern (paid i...   
2                                Country Coordinator   

                                           Company AnnouncementCode Term  \
0             AMERIA Investment Consulting Company              NaN  NaN   
1  International Research & Exchanges Board (IREX)              NaN  NaN   
2        Caucasus Environmental NGO Network (CENN)              NaN  NaN   

  Eligibility Audience StartDate                               Duration  ...  \
0         NaN      NaN       NaN                                    NaN  ...   
1         NaN      NaN       NaN         

## Rename the column to make the data frame more readable

In [17]:
data_selected = data_explore[['jobpost', 'Title', 'Duration', 'Location', 'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary', 'Deadline', 'AboutC', 'date', 'Year', 'Company', 'Month']].copy()

In [18]:
data_selected.columns

Index(['jobpost', 'Title', 'Duration', 'Location', 'JobDescription',
       'JobRequirment', 'RequiredQual', 'Salary', 'Deadline', 'AboutC', 'date',
       'Year', 'Company', 'Month'],
      dtype='object')

In [19]:
data_selected.head()

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

                                             jobpost  \
0  AMERIA Investment Consulting Company\r\nJOB TI...   
1  International Research & Exchanges Board (IREX...   
2  Caucasus Environmental NGO Network (CENN)\r\nJ...   
3  Manoff Group\r\nJOB TITLE:  BCC Specialist\r\n...   
4  Yerevan Brandy Company\r\nJOB TITLE:  Software...   

                                               Title  \
0                            Chief Financial Officer   
1  Full-time Community Connections Intern (paid i...   
2                                Country Coordinator   
3                                     BCC Specialist   
4                                 Software Developer   

                                Duration  \
0                                    NaN   
1                               3 months   
2  Renewable annual contract\r\nPOSITION   
3                                    NaN   
4                                    NaN   

                                            Location  \
0    

In [20]:
column_rename = ['JobPost','JobTitle','JobDuration','JobLocation','JobDescription','JobRequirement','JobQualification','JobSalary','JobDeadline','JobAboutCompany','JobPostDate', 'JobYear', 'JobCompany', 'JobMonth']

In [21]:
data_selected.columns = column_rename

In [22]:
len(data_selected)

18923

In [23]:
data_selected.columns

Index(['JobPost', 'JobTitle', 'JobDuration', 'JobLocation', 'JobDescription',
       'JobRequirement', 'JobQualification', 'JobSalary', 'JobDeadline',
       'JobAboutCompany', 'JobPostDate', 'JobYear', 'JobCompany', 'JobMonth'],
      dtype='object')

In [24]:
data_selected.tail()

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

                                                 JobPost  \
383    "Tire" LLC\r\nTITLE:  Web Programmer & Designe...   
8368   Firmplace Corporation, Yerevan Branch\r\nTITLE...   
7002   ADF Shops CJSC\r\nTITLE:  Shop Assistant\r\nLO...   
7001   "Nork-Marash" Medical Center\r\nTITLE:  Cardio...   
14048  Tumo Center for Creative Technologies\r\nTITLE...   

                              JobTitle JobDuration  \
383          Web Programmer & Designer         NaN   
8368           Clinical Data Associate   Long term   
7002                    Shop Assistant         NaN   
7001                 Cardiology Fellow     2 years   
14048  Instructional Staff Coordinator         NaN   

                                             JobLocation  \
383                                     Yerevan, Armenia   
8368                                    Yerevan, Armenia   
7002                                    Yerevan, Armenia   
7001   Yerevan, Armenia\r\nDETAIL DESCRIPTION:\r\n- K...   
14048         

In [ ]:
data_selected.dtypes

JobPost                     object
JobTitle                    object
JobDuration                 object
JobLocation                 object
JobDescription              object
JobRequirement              object
JobQualification            object
JobSalary                   object
JobDeadline                 object
JobAboutCompany             object
JobPostDate         datetime64[ns]
JobYear                      int64
JobCompany                  object
JobMonth                     int64
dtype: object

In [ ]:
def clean_string(string):
  if string is not None:  
    # Lowercase
    # string = string.lower()

#     # Remove punctuation, tags.
#     regex = re.compile(r'<[^>]*>|\\\'|"|,|\||-|/|\(|\)|\[|\]')
#     string = regex.sub(' ', string)
    # Replace to single space multispaces, newlines, tabs
    regex_space = re.compile(r'\s\s+')
    string = regex_space.sub(' ', string)
  else:
    string = ''
  return string

def remove_html(string):
    parser = HTMLParser()
    string = parser.unescape(string)
    string = re.sub("<.*?>"," ",string)
    string = re.sub("&\w+;"," ",string)
    string = re.sub("\xa0"," ",string)
    string = string.strip()
    return string

def job_title(string):
    restriction = ["title", "duration", "description", "Responsibilities", "qualification", "Remuneration", "deadline", ]
    nlp = spacy.load('en_core_web_sm')
    sentences = nlp(string)
    x = [sent.string.strip() for sent in sentences.sents]
#     job_title = ""
    
#     for sentence in sentences.sents:
#         if 'title' or 'job' in sentence:
#             job_title += sentence
#         elif restriction not in sentence: 
#             continue
    
    return x
        

    
#     for x, word in enumerate(temp):
#         if word[x].casefold() != ("title".casefold()):
#             continue
#         else:
#             while(true):
#                 if word[x].casefold() != ("title".casefold()):
#                 job_title.append(word)
            
            

# data_selected['JobPost'] = data_selected['JobPost'].apply(remove_html)
print(data_selected['JobPost'].apply(job_title))
# data_selected['JobPost'] = data_selected['JobPost'].apply(clean_string)

In [ ]:
a = data_selected['JobPost'][6000]
print(a)
len(a)

In [ ]:
# new data frame with split value columns 
new = data_selected['JobPost'].str.split(":", n = 8, expand = True)

# making seperate first name column from new data frame 
data_selected["a"]= new[0] 
data_selected["b"]= new[1] 
data_selected["c"]= new[2] 
data_selected["d"]= new[3] 
data_selected["e"]= new[4] 
data_selected["f"]= new[5] 
data_selected["g"]= new[6]
data_selected["h"]= new[7]
data_selected["i"]= new[8]

# Dropping old Name columns 
# data_selected.drop(columns =["jobpost"], inplace = True) 

# df display 
data_selected

# Data Checking with provided columns in Dataset

In [ ]:
data_selected[data_selected.JobYear >= 2014].groupby(['JobCompany']).size().reset_index().sort_values([0], ascending = False)

In [22]:
data_selected[data_selected.JobYear >= 2014].groupby(['JobCompany','JobYear']).size().reset_index().sort_values([0,'JobYear'], ascending = False)

,JobCompany,JobYear,0
131,armentel cjsc,2014,45
725,mentor graphics development services cjsc,2014,45
807,orange armenia cjsc,2014,45
242,career center ngo,2015,36
726,mentor graphics development services cjsc,2015,36
284,coca cola hellenic bottling company armenia cjsc,2015,34
132,armentel cjsc,2015,33
1154,world vision armenia,2014,32
616,joomag am llc,2015,30
834,picsart llc,2015,30


In [240]:
# Question c: Identify Company with most number of job ads in past two years 
# mentor graphics development services cjsc

# Question 2
### Identify the month with the highest job ads over the year

In [23]:
data_selected.groupby(['JobMonth']).size()

JobMonth
1     1422
2     1651
3     1692
4     1462
5     1572
6     1658
7     1585
8     1609
9     1648
10    1631
11    1571
12    1420
dtype: int64

### The Answer is March

In [25]:
stop = stopwords.words('english')

In [33]:
def stop_words(string):
  if string is not None:  
    stop_words = set(stopwords.words('english'))   
    word_tokens = word_tokenize(string) 
    string = [w for w in word_tokens if not w in stop_words] 
    string = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
  else:
    string = ''
  return string

In [ ]:
data_selected['JobResponsibilities'] = data_selected['JobResponsibilities'].apply(stop_words)

In [ ]:
a = data_selected["JobPost"][0]
sent_tokenize_list = word_tokenize(a)
print(sent_tokenize_list)